In [ ]:
"""
Segmentation Metrics Evaluation (GT TIFF vs. Prediction TIFF)

What it does
------------
- Reads GT frames from a TIFF (odd pages), applies undefined->3, transposes first N if requested,
  and tracks fully-black frames.
- Reads predicted frames from a multi-page TIFF (output of the prediction script),
  mapping {0,85,170,255} or {0,1,2,3} to class indices {0..3}; detects fully-black pages.
- Skips any frame that is black on either side (union), aligns shapes (transpose GT if needed),
  and computes per-frame and global pixel-wise metrics: IoU / Precision / Recall / F1 for classes {0,1,2,3}.
- Exports ONLY a per-frame CSV named 'frame_metrics_from_folders.csv' to OUT_DIR.

Requirements
------------
Python >= 3.9
pip install numpy opencv-python tifffile scipy pandas scikit-learn tqdm
"""

import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# =========================
# USER CONFIG (EDIT HERE)
# =========================
GT_TIFF_PATH   = "/content/drive/MyDrive/Final_Project/movies/position4_seg_E17.5_231221.tif"   # GT TIFF
PRED_TIFF_PATH = "/content/drive/MyDrive/Final_Project/output_predictions/all_predictions.tif"  # prediction TIFF (multi-page)
OUT_DIR        = "/content/drive/MyDrive/Final_Project/output_predictions"                      # output folder for the CSV

TRANSPOSE_FIRST_N = 1000   # apply cv2.transpose to the first N GT frames (to match your convention)
MAX_FRAMES        = None   # None = use all frames; or int to cap GT frames read

os.makedirs(OUT_DIR, exist_ok=True)

# =========================
# Helpers
# =========================
def is_all_black(u8: np.ndarray) -> bool:
    """
    Check if an image is entirely zeros.

    Args:
        u8 (np.ndarray): grayscale image.

    Returns:
        bool: True if non-empty and all pixels are 0.
    """
    return u8 is not None and u8.size > 0 and (u8.max() == 0)


def png_to_classes(u8: np.ndarray) -> np.ndarray:
    """
    Map a grayscale mask to class indices {0,1,2,3}.
    Accepts either {0,1,2,3} or {0,85,170,255}. Uses robust thresholds.

    Args:
        u8 (np.ndarray): grayscale prediction.

    Returns:
        np.ndarray: class-index map in {0..3}, dtype uint8.
    """
    uniq = np.unique(u8)
    if set(uniq.tolist()).issubset({0, 1, 2, 3}):
        return u8.astype(np.uint8)
    cls = np.zeros_like(u8, dtype=np.uint8)
    cls[u8 >= 213] = 3
    cls[(u8 >= 128) & (u8 < 213)] = 2
    cls[(u8 >=  42) & (u8 < 128)] = 1
    return cls

# =========================
# GT reader (unchanged logic)
# =========================
def read_gt_frames_from_tif(tiff_path: str, transpose_first_n: int = 1000, max_frames=None):
    """
    Read GT from a TIFF (odd pages), apply undefined->3, transpose first N, detect black frames.

    Args:
        tiff_path (str): path to GT TIFF.
        transpose_first_n (int): apply cv2.transpose to the first N frames.
        max_frames (int|None): limit the number of frames read.

    Returns:
        tuple:
            - gt_full (list[Optional[np.ndarray]]): per-frame class maps (0..3) or None if black.
            - black_idx_gt (set[int]): indices of fully black frames.
    """
    import tifffile as tiff
    from scipy.ndimage import binary_dilation

    def separate_undefined_regions(frame: np.ndarray) -> np.ndarray:
        frame = frame.copy()
        zero_mask = (frame == 0)
        cell_mask = (frame == 1) | (frame == 2)
        dilated_cells = binary_dilation(cell_mask)
        undefined_mask = zero_mask & (~dilated_cells)
        frame[undefined_mask] = 3
        return frame

    gt_full = []       # length N: np.ndarray (classes 0..3) or None if black
    black_idx_gt = []  # indices of black frames in the TIFF

    with tiff.TiffFile(tiff_path) as tif:
        all_indices = list(range(1, len(tif.pages), 2))  # masks at odd pages
        if max_frames is not None:
            all_indices = all_indices[:max_frames]

        frame_counter = 0
        for i in tqdm(all_indices, desc="Reading GT from TIFF"):
            frame = tif.pages[i].asarray()
            idx = frame_counter
            if np.all(frame == 0):
                gt_full.append(None)
                black_idx_gt.append(idx)
            else:
                if idx < transpose_first_n:
                    frame = cv2.transpose(frame)
                cleaned = separate_undefined_regions(frame).astype(np.uint8)
                gt_full.append(cleaned)
            frame_counter += 1

    return gt_full, set(black_idx_gt)

# =========================
# Prediction reader (TIFF)
# =========================
def read_pred_from_tif(pred_tif_path: str):
    """
    Read predictions from a multi-page TIFF.

    Args:
        pred_tif_path (str): path to prediction TIFF produced by the prediction script.

    Returns:
        tuple:
            - pred_map (dict[int, Optional[np.ndarray]]): index -> class map (0..3) or None if black.
            - black_idx_pred (set[int]): indices of black prediction frames.
    """
    import tifffile as tiff

    pred_map = {}
    black_idx_pred = set()

    with tiff.TiffFile(pred_tif_path) as tif:
        pages = len(tif.pages)
        if pages == 0:
            raise RuntimeError(f"No images found in {pred_tif_path}")

        for idx in tqdm(range(pages), desc="Reading predictions from TIFF"):
            u8 = tif.pages[idx].asarray()
            if u8 is None:
                continue
            if is_all_black(u8):
                pred_map[idx] = None
                black_idx_pred.add(idx)
            else:
                pred_map[idx] = png_to_classes(u8.astype(np.uint8))

    return pred_map, black_idx_pred

# =========================
# Evaluation (logic preserved)
# =========================
def evaluate_tif_vs_folder(gt_tif=GT_TIFF_PATH, pred_dir=PRED_TIFF_PATH, out_dir=OUT_DIR,
                           transpose_first_n=TRANSPOSE_FIRST_N, max_frames=MAX_FRAMES):
    """
    Compute per-frame and global pixel-wise metrics between GT TIFF and prediction TIFF.

    Prints:
        - Per-class frame-wise means.
        - Global pixel-wise metrics and weighted aggregates.
        - Final accuracy aggregates.

    Side effects:
        - Saves a single CSV 'frame_metrics_from_folders.csv' with one row per frame,
          starting with the four aggregates you requested, followed by per-class metrics.

    Args:
        gt_tif (str): path to GT TIFF.
        pred_dir (str): path to predictions TIFF (multi-page).
        out_dir (str): output directory for the CSV.
        transpose_first_n (int): transpose first N GT frames.
        max_frames (int|None): cap frames read from GT.
    """
    gt_full, black_gt = read_gt_frames_from_tif(gt_tif, transpose_first_n, max_frames)
    pred_map, black_pred = read_pred_from_tif(pred_dir)

    N = len(gt_full)
    print(f"\nTotal frames (by TIFF): {N}")
    print(f"Black in GT (TIFF): {len(black_gt)} | Black in predictions (folder): {len(black_pred)}")

    # Ignore any frame black on either side (union)
    skip = black_gt | black_pred

    # Accumulators
    results = []
    ious_list, precisions_list, recalls_list, f1s_list = [], [], [], []
    gt_all_list, pred_all_list = [], []

    # Valid indices: present in pred_map, not skipped, GT not None
    valid_indices = [i for i in range(N) if (i in pred_map) and (i not in skip) and (gt_full[i] is not None)]
    if not valid_indices:
        raise RuntimeError("No valid pairs to compare (after skipping black frames and unmatched indices).")

    # Iterate all frames (kept as in original loop structure)
    for i in tqdm(range(N), desc="Evaluating"):
        if i not in pred_map:
            continue
        if i in skip:
            continue
        gt = gt_full[i]
        pred = pred_map[i]
        if gt is None or pred is None:
            continue

        # Dimension alignment — transpose GT if exactly reversed
        if gt.shape != pred.shape:
            if gt.shape[::-1] == pred.shape:
                gt = cv2.transpose(gt)
            else:
                raise ValueError(f"Shape mismatch at frame {i}: GT={gt.shape}, Pred={pred.shape}")

        gt_flat   = gt.flatten()
        pred_flat = pred.flatten()

        cm = confusion_matrix(gt_flat, pred_flat, labels=[0, 1, 2, 3])
        inter = np.diag(cm)
        union = np.sum(cm, axis=1) + np.sum(cm, axis=0) - inter
        class_iou = inter / np.maximum(union, 1)

        precision = precision_score(gt_flat, pred_flat, average=None, labels=[0, 1, 2, 3], zero_division=0)
        recall    = recall_score(gt_flat,  pred_flat, average=None, labels=[0, 1, 2, 3], zero_division=0)
        f1        = f1_score(gt_flat,     pred_flat, average=None, labels=[0, 1, 2, 3], zero_division=0)

        # ---- Per-frame aggregates (FIRST in the row) ----
        support_frame = np.bincount(gt_flat, minlength=4)
        weighted_iou_frame = float(np.average(class_iou, weights=support_frame)) if support_frame.sum() > 0 else float("nan")
        weighted_f1_frame  = float(np.average(f1,        weights=support_frame)) if support_frame.sum() > 0 else float("nan")
        cell_macro_iou_fr  = float(np.mean(class_iou[[1, 2]]))
        cell_macro_f1_fr   = float(np.mean(f1[[1, 2]]))

        results.append({
            "frame": i,
            "Cell_Macro_IoU_1_2": cell_macro_iou_fr,
            "Cell_Macro_F1_1_2":  cell_macro_f1_fr,
            "Weighted_IoU_0_3":   weighted_iou_frame,
            "Weighted_F1_0_3":    weighted_f1_frame,
            **{f"IoU_{j}": float(class_iou[j]) for j in range(4)},
            **{f"Precision_{j}": float(precision[j]) for j in range(4)},
            **{f"Recall_{j}": float(recall[j]) for j in range(4)},
            **{f"F1_{j}": float(f1[j]) for j in range(4)},
        })

        ious_list.append(class_iou)
        precisions_list.append(precision)
        recalls_list.append(recall)
        f1s_list.append(f1)

        gt_all_list.append(gt_flat)
        pred_all_list.append(pred_flat)

    if not results:
        raise RuntimeError("No results computed. Check file naming/indexing and black-frame handling.")

    # Frame-wise means (prints kept)
    ious_arr       = np.array(ious_list)
    precisions_arr = np.array(precisions_list)
    recalls_arr    = np.array(recalls_list)
    f1s_arr        = np.array(f1s_list)

    print("\n📊 Averaged Metrics (frame-wise means):")
    for j in range(4):
        print(f"Class {j} - IoU: {np.nanmean(ious_arr[:, j]):.4f}, "
              f"Precision: {np.nanmean(precisions_arr[:, j]):.4f}, "
              f"Recall: {np.nanmean(recalls_arr[:, j]):.4f}, "
              f"F1: {np.nanmean(f1s_arr[:, j]):.4f}")

    # Global (pixel-wise over all used frames) — prints kept
    gt_all   = np.concatenate(gt_all_list, axis=0)
    pred_all = np.concatenate(pred_all_list, axis=0)

    cm_global = confusion_matrix(gt_all, pred_all, labels=[0, 1, 2, 3])
    support   = np.bincount(gt_all, minlength=4)
    inter_g   = np.diag(cm_global)
    union_g   = np.sum(cm_global, axis=1) + np.sum(cm_global, axis=0) - inter_g
    class_iou_g = inter_g / np.maximum(union_g, 1)

    precision_g = precision_score(gt_all, pred_all, average=None, labels=[0, 1, 2, 3], zero_division=0)
    recall_g    = recall_score(gt_all,  pred_all, average=None, labels=[0, 1, 2, 3], zero_division=0)
    f1_g        = f1_score(gt_all,     pred_all, average=None, labels=[0, 1, 2, 3], zero_division=0)

    print("\n📊 Global Pixel-wise Metrics:")
    for j in range(4):
        print(f"Class {j} - IoU: {class_iou_g[j]:.4f} (support: {int(support[j])})")

    weighted_mean_iou = np.average(class_iou_g, weights=support)
    weighted_precision = np.average(precision_g, weights=support)
    weighted_recall = np.average(recall_g, weights=support)
    weighted_f1 = np.average(f1_g, weights=support)

    print(f"\nWeighted Mean IoU:   {weighted_mean_iou:.4f}")
    print(f"Weighted Precision:  {weighted_precision:.4f}")
    print(f"Weighted Recall:     {weighted_recall:.4f}")
    print(f"Weighted F1 Score:   {weighted_f1:.4f}")

    # Final accuracy aggregates (prints kept)
    cell_macro_iou = float(np.mean(class_iou_g[[1, 2]]))
    cell_macro_f1  = float(np.mean(f1_g[[1, 2]]))

    print("\n=== Final Accuracy (as reported in the project) ===")
    print(f"Cell Macro IoU (classes 1–2): {cell_macro_iou:.4f}")
    print(f"Cell Macro F1  (classes 1–2): {cell_macro_f1:.4f}")
    print(f"Weighted IoU (0–3):           {weighted_mean_iou:.4f}")
    print(f"Weighted F1  (0–3):           {weighted_f1:.4f}")

    # Save ONLY the per-frame CSV (first columns: the four aggregates you requested)
    df = pd.DataFrame(results).sort_values("frame")
    csv_path = os.path.join(out_dir, "frame_metrics_from_folders.csv")
    df.to_csv(csv_path, index=False)
    print(f"\n✅ Saved per-frame CSV to: {csv_path}")


if __name__ == "__main__":
    evaluate_tif_vs_folder()


Reading predictions from TIFF: 100%|██████████| 273/273 [00:26<00:00, 10.43it/s]



Total frames (by TIFF): 273
Black in GT (TIFF): 52 | Black in predictions (folder): 52


Evaluating: 100%|██████████| 273/273 [06:40<00:00,  1.47s/it]



📊 Averaged Metrics (frame-wise means):
Class 0 - IoU: 0.6675, Precision: 0.7225, Recall: 0.8972, F1: 0.8003
Class 1 - IoU: 0.8793, Precision: 0.9429, Recall: 0.9282, F1: 0.9353
Class 2 - IoU: 0.8913, Precision: 0.9183, Recall: 0.9678, F1: 0.9414
Class 3 - IoU: 0.3213, Precision: 0.7708, Recall: 0.3372, F1: 0.4373

📊 Global Pixel-wise Metrics:
Class 0 - IoU: 0.6672 (support: 33884291)
Class 1 - IoU: 0.8793 (support: 180931758)
Class 2 - IoU: 0.8901 (support: 544693430)
Class 3 - IoU: 0.4015 (support: 65400425)

Weighted Mean IoU:   0.8398
Weighted Precision:  0.9144
Weighted Recall:     0.9127
Weighted F1 Score:   0.9055

=== Final Accuracy (as reported in the project) ===
Cell Macro IoU (classes 1–2): 0.8847
Cell Macro F1  (classes 1–2): 0.9388
Weighted IoU (0–3):           0.8398
Weighted F1  (0–3):           0.9055

✅ Saved per-frame CSV to: /content/drive/MyDrive/Final_Project/output_predictions/frame_metrics_from_folders.csv
